# ExamplesOfUNITY

In [1]:
import sys
sys.path.append("../")
from IoTPy.core.stream import Stream, run, _no_value
from IoTPy.agent_types.op import signal_element, map_element
from IoTPy.agent_types.merge import weave_f
from IoTPy.helper_functions.recent_values import recent_values

In [2]:
def sort(lst):
    """
    Notes
    -----
    The program has len(lst)-1 agents indexed i = 1, 2, .. len(lst)-1.
    It has len(lst)+1 streams indexed j = 0, 1, ..., len(lst).
    The content of each stream is a sequence of 1s. Each 1 is a signal
    that an action needs to be executed. The action is on a shared
    data structure which, in this example, is list, called lst.

    agent i is responsible for ensuring lst[i-1] <= lst[i], and this
    agent may modify lst[i-1] and lst[i]. So agent i listens for a
    signal from agent i+1 because agent i+1 may modify lst[i]. Also,
    agent i listens for a signal from agent i-1 because agent i-1 may
    modify lst[i-1]. So, agent i listens to a weave of the signals
    generated by agents i-1 and i+1.

    """
    def flip(i):
        if lst[i-1] > lst[i]:
            lst[i-1], lst[i] = lst[i], lst[i-1]
            # Output a signal because lst has changed.
            return 1
        else:
            # Do not output a signal because lst remained unchanged.
            return _no_value
    # Create streams
    S = [Stream() for i in range(len(lst)+1)]
    # Create agents
    for i in range(1, len(lst)):
        signal_element(
            func=flip, in_stream=weave_f([S[i-1],S[i+1]]),
            out_stream=S[i], i=i)
    # Initiate the algorithm by putting a signal on each stream.
    for stream in S: stream.append(1)
    run()

In [3]:
lst = [2, 3, 1, 0]
sort(lst)
print ('lst is ', lst)

lst is  [0, 1, 2, 3]


In [4]:
def shortest_path(D):
    """
    Parameters
    ----------
    D: matrix where D[j,k] is the length of the edge from vertex j to
    vertex k.

    Returns
    -------
    D: matrix where D[j,k] is the length of the shortest path from
    vertex j to  vertex k.
    
    """
    #----------------------------------------------------------------
    # STEP 1. DEFINE FUNCTION TO BE ENCAPSULATED
    def triangle_inequality(triple):
        """
        Apply the triangle inequality. If this changes D then
        return any value (1 in our example). If D is unchanged
        then return no value.

        Parameters
        ----------
        triple: 3-element array or list

        """
        i, j, k = triple
        if D[i][j] + D[j][k] < D[i][k]:
            D[i][k] = D[i][j] + D[j][k]
            # Output a signal because D has changed.
            return 1
        else:
            # Do not output a signal because D remained unchanged.
            return _no_value

    #----------------------------------------------------------------
    # STEP 2. CREATE STREAMS
    # Create an array, changed, of streams, where a value is appended
    # to changed[i][k] when D[i][k] is changed.
    indices = range(len(D))
    changed = [[ Stream('changed_'+ str(i)+"-" + str(j))
                 for i in indices] for j in indices]

    #----------------------------------------------------------------
    # STEP 3. CREATE AGENTS
    # Create an agent for each triple i,j,k. The agent executes its
    # action when it reads a new element of stream x. If it changes D
    # it then puts a new element on x.
    for i in indices:
        for j in indices:
            for k in indices:
                signal_element(func=triangle_inequality,
                               in_stream=weave_f([changed[i][j], changed[j][k]]),
                               out_stream=changed[i][k],
                               triple=[i, j, k])

    #----------------------------------------------------------------
    #STEP 4. START COMPUTATION
    # Start the computation by putting a signal on stream changed[i,j].
    for i in indices:
        for j in indices:
            changed[i][j].append(1)
    run()
    
    return D

In [5]:
# EXAMPLE 2: MATRIX OF LENGTHS OF SHORTEST PATHS
D = [[0, 20, 40, 60], [20, 0, 10, 1], [40, 10, 0, 100], [60, 1, 100, 0]]
shortest_path(D)
assert D == [[0, 20, 30, 21], [20, 0, 10, 1], [30, 10, 0, 11], [21, 1, 11, 0]]

# The next example illustrates call_streams

In [6]:
x = Stream()
y = Stream()
z = Stream()
b = map_element(lambda v: 10*v, x, y, call_streams=[z])

x.extend([1])
run()
print ('step 1')
print (recent_values(z), 'z')
print (recent_values(y), 'y')

z.extend([1])
run()
print ('step 2')
print (recent_values(z), 'z')
print (recent_values(y), 'y')

z.extend([1])
run()
print ('step 3')
print (recent_values(z), 'z')
print (recent_values(y), 'y')

x.extend([2])
run()
print ('step 4')
print (recent_values(z), 'z')
print (recent_values(y), 'y')

z.extend([1])
run()
print ('step 5')
print (recent_values(z), 'z')
print (recent_values(y), 'y')



step 1
[] z
[] y
step 2
[1] z
[10] y
step 3
[1, 1] z
[10] y
step 4
[1, 1] z
[10] y
step 5
[1, 1, 1] z
[10, 20] y
